# 1. Gerekli kütüphaneleri yükle

In [ ]:
!pip install huggingface_hub torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.5 MB/s eta 0:00:00


# 2. Kütüphaneleri import et

In [ ]:
from huggingface_hub import hf_hub_download
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. src klasörünü Colab'a yüklemen gerekiyor
Bunu Colab arayüzünden manuel yap veya Google Drive bağla Sonra yolunu sys.path'e ekle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/src')

# 4. Modellerin tanımlarını import et

In [ ]:
from wgan_gp import Generator, Critic
from flow_module_real import RealNVP
from transformer_fc import TransformerForecast

# 5. Hugging Face repo ID ve dosya isimleri

In [ ]:
repo_id = "Bengsucakmak/anomalyflowgan-models"

critic_path = hf_hub_download(repo_id=repo_id, filename="critic.pth")
generator_path = hf_hub_download(repo_id=repo_id, filename="generator.pth")
realnvp_path = hf_hub_download(repo_id=repo_id, filename="realnvp_real.pth")
transformer_path = hf_hub_download(repo_id=repo_id, filename="transformer_fc.pth")

print("Model dosyaları indirildi.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


critic.pth:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

generator.pth:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

realnvp_real.pth:   0%|          | 0.00/39.4M [00:00<?, ?B/s]

transformer_fc.pth:   0%|          | 0.00/594k [00:00<?, ?B/s]

Model dosyaları indirildi.


# 6. Modelleri oluştur ve ağırlıkları yükle

In [ ]:
critic = Critic().to(device)
critic.load_state_dict(torch.load(critic_path, map_location=device))
critic.eval()

generator = Generator().to(device)
generator.load_state_dict(torch.load(generator_path, map_location=device))
generator.eval()

input_dim = 200 * 4  # flow_module_real.py'deki ayarlara göre
num_coupling_layers = 6

realnvp = RealNVP(dim=input_dim, n_coupling_layers=num_coupling_layers).to(device)
realnvp.load_state_dict(torch.load(realnvp_path, map_location=device))
realnvp.eval()



input_dim = 4       # transformer_fc.py NUM_CHANNELS
seq_len = 150       # transformer_fc.py ENC_LEN
output_dim = 4      # transformer_fc.py NUM_CHANNELS

transformer = TransformerForecast().to(device)
transformer.load_state_dict(torch.load(transformer_path, map_location=device))
transformer.eval()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


TransformerForecast(
  (input_proj): Linear(in_features=4, out_features=64, bias=True)
  (pos_enc): PositionalEncoding()
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=200, bias=True)
  )
)

# 7. Örnek inference (tahmin) yap

In [ ]:
with torch.no_grad():
    # WGAN-GP Generator örneği
    latent_dim = 100
    z = torch.randn(5, latent_dim).to(device)
    gen_samples = generator(z)
    print("Generated samples shape:", gen_samples.shape)

    # RealNVP Anomaly Scoring örneği
    SEQ_LEN = 200      # flow_module_real.py’daki sabit
    input_dim = 4      # NUM_CHANNELS
    example_data = torch.tensor(np.random.randn(5, SEQ_LEN * input_dim), dtype=torch.float32).to(device)
    log_prob = realnvp.log_prob(example_data)
    anomaly_scores = -log_prob
    print("Anomaly scores:", anomaly_scores.cpu().numpy())

    # Transformer Forecast örneği
    seq_len = 150
    input_dim = 4
    example_seq = torch.tensor(np.random.randn(1, seq_len, input_dim), dtype=torch.float32).to(device)
    forecast_output = transformer(example_seq)
    print("Forecast output shape:", forecast_output.shape)
    print("Forecast output:", forecast_output.cpu().numpy())


Generated samples shape: torch.Size([5, 200, 4])
Anomaly scores: [ 91641.836 134738.98  125552.234 110567.18   73109.805]
Forecast output shape: torch.Size([1, 50, 4])
Forecast output: [[[-0.34343532 -0.07408784  0.7385761  -0.9876906 ]
  [ 0.22067863  0.13785817  0.24938916 -0.4936557 ]
  [ 0.5969829   0.22864018 -0.85657775 -0.19693828]
  [-0.10186264  0.44731885 -0.63805187  0.56972957]
  [-0.62385756  0.7149088  -0.11729082 -0.18253955]
  [-0.2960865   0.35770035  0.17619114 -0.6554176 ]
  [ 0.3926603   0.3436679  -0.40905273 -0.7079692 ]
  [ 0.4575665  -0.02898709 -0.63373476 -0.13151446]
  [ 0.17905673  0.2634361  -0.11004132 -0.13944326]
  [-0.06134676 -0.14662376  0.45763135 -0.10117436]
  [ 0.02998821 -0.21295552 -0.06631391 -0.03057136]
  [-0.37153417  0.08735756 -0.3934087   0.306471  ]
  [-0.56402326  0.00928862 -0.3472794   0.0115712 ]
  [-0.24130604 -0.14230187  0.05663548 -0.24197806]
  [-0.2537002  -0.47465706  0.47441825 -0.18178678]
  [-0.22657427  0.12701097  0.20326